In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
import copy
from tqdm.notebook import tqdm_notebook
import math
from skopt import gbrt_minimize
from skopt.space import Real
import warnings
warnings.filterwarnings('ignore')

device = torch.device("mps" if torch.has_mps else "cpu")
print(device)
from itertools import product
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

random_state = 1

mps


In [2]:
# read in data and split
X = torch.load('inputs/bfw_resnet50_face_embeddings.pt').cpu()
y = torch.load('inputs/bfw_resnet50_labels.pt').cpu()
df = pd.read_csv('inputs/bfw_resnet50_df.csv')
gender = df['reference_gender']
df

,reference_identity,candidate_identity,reference_ethnicity,candidate_ethnicity,reference_gender,candidate_gender,labels
0,n004721,n004721,asian,asian,male,male,1.0
1,n004721,n004721,asian,asian,male,male,1.0
2,n004721,n004721,asian,asian,male,male,1.0
3,n004721,n004721,asian,asian,male,male,1.0
4,n004721,n004721,asian,asian,male,male,1.0
...,...,...,...,...,...,...,...
38395,n003412,n005685,white,white,male,male,0.0
38396,n008917,n008839,black,black,female,female,0.0
38397,n008452,n008150,asian,asian,male,male,0.0
38398,n004936,n002509,asian,asian,female,female,0.0


In [3]:
train_split, test_split = train_test_split(np.arange(len(X)),test_size=0.2, random_state=random_state)
train_split, val_split = train_test_split(train_split,test_size=0.25, random_state=random_state)
X_train = X[train_split]
X_val = X[val_split]
X_test = X[test_split]
y_train = y[train_split]
y_val = y[val_split]
y_test = y[test_split]

gender_train = gender[train_split].values
gender_train[gender_train=='male'] = 0
gender_train[gender_train=='female'] = 1
gender_train = gender_train.astype(int)

gender_val = gender[val_split].values
gender_val[gender_val=='male'] = 0
gender_val[gender_val=='female'] = 1
gender_val = gender_val.astype(int)

gender_test = gender[test_split].values
gender_test[gender_test=='male'] = 0
gender_test[gender_test=='female'] = 1
gender_test = gender_test.astype(int)

In [4]:
train_df = df.iloc[train_split]
val_df = df.iloc[val_split]
test_df = df.iloc[test_split]
test_df

,reference_identity,candidate_identity,reference_ethnicity,candidate_ethnicity,reference_gender,candidate_gender,labels
340,n005178,n005178,asian,asian,male,male,1.0
18908,n001229,n001229,white,white,female,female,1.0
37816,n008972,n003463,asian,asian,male,male,0.0
10524,n000138,n000138,asian,asian,female,female,1.0
7492,n003398,n003398,white,white,male,male,1.0
...,...,...,...,...,...,...,...
7045,n007184,n007184,indian,indian,male,male,1.0
10213,n004472,n004472,asian,asian,female,female,1.0
12000,n002274,n002274,black,black,female,female,1.0
36114,n000660,n008636,white,white,female,female,0.0


In [5]:
## train data
class TrainData(data.Dataset):
    
    def __init__(self, X_data, y_data,gender):
        self.X_data = X_data
        self.y_data = y_data
        self.gender = gender    
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index], self.gender[index]
        
    def __len__ (self):
        return len(self.X_data)


train_data = TrainData(torch.FloatTensor(X_train), 
                       torch.FloatTensor(y_train),
                       gender_train)
test_data = TrainData(torch.FloatTensor(X_test),torch.FloatTensor(y_test),gender_test)
val_data = TrainData(torch.FloatTensor(X_val), 
                       torch.FloatTensor(y_val),
                       gender_val)
class BinaryClassification(nn.Module):
    def __init__(self):
        super(BinaryClassification, self).__init__()
        # Number of input features is 4096
        self.layer_1 = nn.Linear(4096, 1024) 
        self.layer_2 = nn.Linear(1024, 512)
        self.layer_out = nn.Linear(512, 1) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(1024)
        self.batchnorm2 = nn.BatchNorm1d(512)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x

def confusion_mat(y_pred, y_test):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    acc = (tn + tp)/(tn+tp+fn+fp)
    return tn, fp, fn, tp, acc
    


In [22]:
EPOCHS = 20
BATCH_SIZE = 64
LEARNING_RATE = 1e-5
eps = 1e-5
model = torch.load('weights/bfw_senet50_logistic_regression_face_matching_0.0_betaTEST.pt')
model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

train_loader = data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = data.DataLoader(dataset=test_data, batch_size=BATCH_SIZE)
val_loader = data.DataLoader(dataset=val_data, batch_size=BATCH_SIZE)


In [23]:
def val_model(model, loader, criterion):
    """Validate model on loader with criterion function"""
    y_true, y_pred, y_prot = [], [], []
    model.eval()
    with torch.no_grad():
        for inputs, labels,protected in loader:
            # inputs = inputs.to(device)
            y_true.append(labels)
            y_prot.append(protected)
            y_pred.append(torch.sigmoid(model(inputs)).cpu())
    y_true, y_pred, y_prot = torch.cat(y_true), torch.cat(y_pred), torch.cat(y_prot)
    return criterion(y_true, y_pred, y_prot)
def compute_bias(y_pred, y_true, prot, metric):
    """Compute bias on the dataset"""
    def zero_if_nan(data):
        """Zero if there is a nan"""
        return 0. if torch.isnan(data) else data

    gtpr_prot = zero_if_nan(y_pred[prot * y_true == 1].mean())
    gfpr_prot = zero_if_nan(y_pred[prot * (1-y_true) == 1].mean())
    mean_prot = zero_if_nan(y_pred[prot == 1].mean())

    gtpr_unprot = zero_if_nan(y_pred[(1-prot) * y_true == 1].mean())
    gfpr_unprot = zero_if_nan(y_pred[(1-prot) * (1-y_true) == 1].mean())
    mean_unprot = zero_if_nan(y_pred[(1-prot) == 1].mean())

    if metric == "spd":
        return mean_prot - mean_unprot
    elif metric == "aod":
        return 0.5 * ((gfpr_prot - gfpr_unprot) + (gtpr_prot - gtpr_unprot))
    elif metric == "eod":
        return gtpr_prot - gtpr_unprot
    elif 'false_diff':
        return (np.abs(gfpr_prot - gfpr_unprot) + np.abs(gtpr_unprot - gtpr_prot))

def compute_objective(performance, bias, epsilon=0.001, margin=0.0005):
    if abs(bias) <= (epsilon-margin):
        return performance
    else:
        return 0.0
def get_best_objective(y_true, y_pred, y_prot):
    """Find the threshold for the best objective"""
    num_samples = 5
    threshs = torch.linspace(0, 1, 501)
    best_obj, best_thresh = -math.inf, 0.
    for thresh in threshs:
        indices = np.random.choice(np.arange(y_pred.size()[0]), num_samples*y_pred.size()[0], replace=True).reshape(num_samples, y_pred.size()[0])
        objs = []
        for index in indices:
            y_pred_tmp = y_pred[index]
            y_true_tmp = y_true[index]
            y_prot_tmp = y_prot[index]
            perf = (torch.mean((y_pred_tmp > thresh)[y_true_tmp.type(torch.bool)].type(torch.float32)) + torch.mean((y_pred_tmp <= thresh)[~y_true_tmp.type(torch.bool)].type(torch.float32))) / 2
            bias = compute_bias((y_pred_tmp > thresh).float().cpu(), y_true_tmp.float().cpu(), y_prot_tmp.float().cpu(), 'false_diff')
            objs.append(compute_objective(perf, bias))
        obj = float(torch.tensor(objs).mean())
        if obj > best_obj:
            best_obj, best_thresh = obj, thresh

    return best_obj, best_thresh

In [24]:
# compute bias before applying random processing
y_true, y_pred, y_prot = [], [], []
model.eval()
with torch.no_grad():
    for inputs, labels,protected in val_loader:
        inputs = inputs.to(device)
        y_true.append(labels)
        y_prot.append(protected)
        y_pred.append(torch.sigmoid(model(inputs)).cpu())
y_true, y_pred, y_prot = torch.cat(y_true), torch.cat(y_pred), torch.cat(y_prot)

compute_bias((y_pred>0.5).float().cpu(), y_true, y_prot, 'false_diff')


tensor(0.0132)

In [47]:
  unrefined_net = get_resnet_model()
        base_model = copy.deepcopy(unrefined_net)
        base_model.fc = nn.Linear(base_model.fc.in_features, base_model.fc.in_features)

        actor = nn.Sequential(base_model, nn.Linear(base_model.fc.in_features, 2))
        actor.to(device)
        actor_optimizer = optim.Adam(actor.parameters(), lr=1e-4)
        actor_loss_fn = nn.BCEWithLogitsLoss()
        actor_loss = 0.
        actor_steps = config['adversarial']['actor_steps']

        critic = Critic(config['batch_size']*unrefined_net.fc.in_features)
        critic.to(device)
        critic_optimizer = optim.Adam(critic.parameters(), lr=1e-4)
        critic_loss_fn = nn.MSELoss()
        critic_loss = 0.
        critic_steps = config['adversarial']['critic_steps']

        for epoch in range(config['adversarial']['epochs']):
            for param in critic.parameters():
                param.requires_grad = True
            for param in actor.parameters():
                param.requires_grad = False
            actor.eval()
            critic.train()
            for step, (inputs, labels) in enumerate(valloader):
                if step > critic_steps:
                    break
                inputs, labels = inputs.to(device), labels.to(device)
                if inputs.size(0) != config['batch_size']:
                    continue
                critic_optimizer.zero_grad()

                with torch.no_grad():
                    y_pred = actor(inputs)

                y_true = labels[:, prediction_index].float().to(device)
                y_prot = labels[:, protected_index].float().to(device)

                bias = compute_bias(y_pred, y_true, y_prot, config['metric'])
                res = critic(base_model(inputs))
                loss = critic_loss_fn(bias.unsqueeze(0), res[0])
                loss.backward()
                critic_loss += loss.item()
                critic_optimizer.step()
                if step % 100 == 0:
                    print_loss = critic_loss if (epoch*critic_steps + step) == 0 else critic_loss / (epoch*critic_steps + step)
                    logger.info(f'=======> Epoch: {(epoch, step)} Critic loss: {print_loss:.3f}')

            for param in critic.parameters():
                param.requires_grad = False
            for param in actor.parameters():
                param.requires_grad = True
            actor.train()
            critic.eval()
            for step, (inputs, labels) in enumerate(valloader):
                if step > actor_steps:
                    break
                inputs, labels = inputs.to(device), labels.to(device)
                if inputs.size(0) != config['batch_size']:
                    continue
                actor_optimizer.zero_grad()

                y_true = labels[:, prediction_index].float().to(device)
                y_prot = labels[:, protected_index].float().to(device)

                est_bias = critic(base_model(inputs))
                loss = actor_loss_fn(actor(inputs)[:, 0], y_true)

                loss = max(1, config['adversarial']['lambda']*(abs(est_bias)-config['objective']['epsilon']+config['adversarial']['margin'])+1) * loss

                loss.backward()
                actor_loss += loss.item()
                actor_optimizer.step()
                if step % 100 == 0:
                    print_loss = critic_loss if (epoch*actor_steps + step) == 0 else critic_loss / (epoch*actor_steps + step)
                    logger.info(f'=======> Epoch: {(epoch, step)} Actor loss: {print_loss:.3f}')

            print_objective_results(valloader, actor, best_thresh, protected_index, prediction_index)
        _, best_thresh = val_model(actor, valloader, get_best_objective, protected_index, prediction_index)

        logger.info('val_results')
        valid_results['adversarial'] = print_objective_results(valloader, actor, best_thresh, protected_index, prediction_index)
        logger.info('test_results')
        test_results['adversarial'] = print_objective_results(testloader, actor, best_thresh, protected_index, prediction_index)


  0%|          | 0/10 [00:00<?, ?it/s]

Evaluating param number 0 of 10
Number of sparse indices: 4906
Evaluating param number 1 of 10
Number of sparse indices: 1024
Evaluating param number 2 of 10
Number of sparse indices: 5001
Evaluating param number 3 of 10
Number of sparse indices: 512
Evaluating param number 4 of 10
Number of sparse indices: 512
Evaluating param number 5 of 10
Number of sparse indices: 1
Evaluating param number 6 of 10
Number of sparse indices: 1024
Evaluating param number 7 of 10
Number of sparse indices: 1024
Evaluating param number 8 of 10
Number of sparse indices: 512
Evaluating param number 9 of 10
Number of sparse indices: 512


tensor(0.)

In [10]:
compute_bias((y_pred>best_thresh).float().cpu(), y_true, y_prot, 'aod')

tensor(0.)